# Hypothesis: _Property_ -basiertes Testen

In diesem Notebook verwenden wir _Property_ -basierte Tests, um Probleme in unserem Code zu finden. [Hypothesis](https://hypothesis.readthedocs.io/en/latest/) ist eine Bibliothek, die Haskells [Quickcheck](https://hackage.haskell.org/package/QuickCheck) ähnelt. Später lernen wir sie zusammen mit anderen Testbibliotheken noch genauer kennen: [Hypothesis](https://jupyter-tutorial.readthedocs.io/de/latest/notebook/testing/hypothesis.html). Hypothesis kann auch Mock-Objekte und Tests für Numpy-Datentypen bereitstellen.

## 1. Importe

In [1]:
import re

from hypothesis import assume, given
from hypothesis.strategies import emails, integers, tuples                                                                                                       

## 2. Bereich finden

In [2]:
def calculate_range(tuple_obj):
    return max(tuple_obj) - min(tuple_obj)

## 3. Test mit `strategies` und `given`

Mit [hypothesis.strategies](https://hypothesis.readthedocs.io/en/latest/data.html) könnt ihr unterschiedliche Testdaten erstellen. Hierfür beitet Hypothesis Strategien für die meisten Typen und Argumente schränken die Möglichkeiten ein um sie euren Erfordernissen anzupassen. Im Beispiel unten verwenden wir die [integers](https://hypothesis.readthedocs.io/en/latest/data.html#hypothesis.strategies.integers)-Strategie, die mit dem [Python-Decorator](https://docs.python.org/3/glossary.html#term-decorator) `@given` auf die Funktion angewendet wird. Genauer  nimmt er unsere Testfunktion und wandelt sie in eine parametrisierte um sie über weite Bereiche passender Daten auszuführen:

In [3]:
@given(tuples(integers(), integers(), integers()))
def test_calculate_range(tup):
    result = calculate_range(tup)
    assert isinstance(result, int)
    assert result > 0

In [4]:
test_calculate_range()

AssertionError: 

Nun korrigieren wir den Test mit `>=` und überprüfen ihn erneut:

In [5]:
@given(tuples(integers(), integers()))
def test_calculate_range(tup):
    result = calculate_range(tup)
    assert isinstance(result, int)
    assert result >= 0

In [6]:
test_calculate_range()

## 3. Gegen Reguläre Ausdrücke prüfen

Mit [regulären Ausrücken](https://de.wikipedia.org/wiki/Regul%C3%A4rer_Ausdruck) (engl.: _regular expressions_) lassen sich Zeichenketten auf bestimmte syntaktische Regeln überprüfen. In Python könnt ihr zum Überprüfen regulärer Ausdrücke [re.match](https://docs.python.org/3/library/re.html#re.match) verwenden.

> **Hinweis:**
> 
> Auf der Website [regex101](https://regex101.com/) könnt ihr zunächst eure regulären Ausdrücke ausprobieren.

Als Beispiel versuchen wir, aus E-Mail-Adressen `username` und die `domain` zu ermitteln:

In [7]:
def parse_email(email):
    result = re.match("(?P<username>\w+).(?P<domain>[\w\.]+)", email).groups()
    return result

Nun schreiben wir einen Test `test_parse_email` zum Überprüfen unserer Methode. Als Eingabewerte verwenden wir die [emails](https://hypothesis.readthedocs.io/en/latest/data.html#hypothesis.strategies.emails)-Strategie von Hypothesis. Als `result` erwarten wir z.B.:
```
('0', 'A.com')
('F', 'j.EeHNqsx')
…
```
Im Test nehmen wir einerseits an, dass immer zwei Einträge zurückgegeben werden und im zweiten Eintrag ein Punkt (`.`) vorkommt. 

In [8]:
@given(emails())
def test_parse_email(email):
    result = parse_email(email)
    # print(result)
    assert len(result) == 2
    assert '.' in result[1]

In [9]:
test_parse_email()

ExceptionGroup: Hypothesis found 2 distinct failures. (2 sub-exceptions)

Mit Hypothesis wurden zwei Beispiele gefunden, die deutlich machen, dass unser regulärer Ausdruck in der `parse_email`-Methode noch nicht hinreichend ist: `0/0@A.ac` und `/@A.ac`. Nachdem wir unseren regulären Ausdruck entsprechend angepasst haben, können wir den Test erneut aufrufen:

In [10]:
def parse_email(email):
    result = re.match(
        "(?P<username>[\.\w\-\!~#$%&\|{}\+\/\^\`\=\*']+).(?P<domain>[\w\.\-]+)",
        email,
    ).groups()
    return result

In [11]:
test_parse_email()